In [13]:
from tqdm.notebook import tqdm
import pandas as pd
from pygpt4all.models.gpt4all_j import GPT4All_J
import time 
from datetime import datetime

In [14]:


def getanswer(question):
    def new_text_callback(text):
        print(text, end='')
        return text
    model = GPT4All_J('/Users/aaronsteiner/Downloads/ggml-gpt4all-j-v1.2-jazzy.bin')
    response = model.generate(question, seed=42069, new_text_callback=new_text_callback)
    return response.replace(question, '').strip()

In [15]:
getanswer("Are these two products the same? product 1: TAG Heuer Monaco Chronograph Calibre 11 Automatic, product 2: TAG Heuer Men's Special Edition Heuer Monaco Watch Chrono Grapple 11 Automatic")

gptj_model_load: loading model from '/Users/aaronsteiner/Downloads/ggml-gpt4all-j-v1.2-jazzy.bin' - please wait ...
Are these two products the same? product 1gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 4505.45 MB
gptj_model_load: memory_size =   896.00 MB, n_mem = 57344
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285
gptj_generate: seed = 42069
gptj_generate: number of tokens in prompt = 47

: TAG Heuer Monaco Chronograph Calibre 11 Automatic, product 2: TAG Heuer Men's Special Edition Heuer Monaco Watch Chrono Grapple 11 Automatic
Yes, both products are from TAG Heuer and they both have the model number 11. The difference is in the materials used for the watchband and dial, as the first product is made of titaniu

'Yes, both products are from TAG Heuer and they both have the model number 11. The difference is in the materials used for the watchband and dial, as the first product is made of titanium and the second is made of titanium alloy. Both watches have a 44 mm case and an 18k gold plated crown.<|endoftext|>'

## Start Testing

In [16]:
# open our test dataframe from the csv
df = pd.read_csv('data/test/random_sample_100.csv')
df.head()

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,...,priceCurrency_right,specTableContent_right,cluster_id_right,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction
0,1639,58378163,NaN,Brother HL-L6200DW Wireless High Speed Mono Laser 2-sided printing 250 sheet Network 46PPM,"WIRELESS HIGH SPEED MONO LASER PRINTER WITH2-Sided PRINTING FOR HIGH VOLUME USAGE(46 PPM, 520 Sheets Paper Tray, Built-in Network & WiFi)A monochrome printer with great features• Print using built-in wireless, Gigabit Ethernet networking or locally via USB interface.• Prints up to 48 pages per minute at up to 1200 x 1200 dpi resolution.• 520-sheet capacity paper tray adjusts for letter or legal size paper.50-sheet multi-purpose tray is for envelopes or alternative media types.• Save paper with automatic duplex printing (double-sided).• Advanced security restricts unauthorized printing and protects sensitive documents.• Easy to use control panel with backlit LCD display for easy",617.38,AUD,NaN,5712817,44664998,...,USD,NaN,408446,58378163#44664998,0,True,"[3.8065164089, -4.2674832344]",0,[0.0],0
1,460,91664569,NaN,Double Socket Arm - Long - RAM-B-201U-C,"Long Arm is 6\"" for applications requiring more reach.",23.49,USD,NaN,1366684,17589843,...,USD,NaN,2490160,91664569#17589843,0,True,"[3.8207249641, -4.2826638222]",0,[0.0],0
2,1760,1575940,NaN,Corsair Vengeance LPX RAM 16GB 2400MHz DDR4 UDIMM 288 Pin Desktop Memory,"CORSAIR Vengeance LPX 16GB DDR4 memory is designed for high-performance overclocking. Built from an custom performance PCB and highly-screened memory ICs.The heatspreader is made of pure aluminum for faster heat dissipation and the custom performance PCB helps manage heat and provides superior overclocking headroom.Vengeance modules are compatibility-tested across DDR4 systems for reliably fast performance. The small form factor makes it ideal for smaller cases or any system where internal space is at a premium.Support XMP 2.0, one setting is all it takes to automatically adjust to the fastest safe speed for your Vengeance LPX kit.~section~High-performance OverclockingVENGEANCE LPX memory is designed",5499.00,INR,NaN,150984,31345669,...,NaN,NaN,150984,1575940#31345669,1,False,"[-4.3838095665, 4.6694221497]",1,[1.0],1
3,4145,7125489,NaN,"HIK 8CH TURBO HD DVR, DS-7208HUHI-K1","Hikvision 8-Channel Turbo HD DVR, 1080p, 3MP, 5MP, 8MP, BNC Spot out, VGA/HDMI, up to 16ch IP",0.00,NaN,NaN,1069672,50492036,...,EUR,NaN,56750,7125489#50492036,0,True,"[3.8441700935, -4.3036942482]",0,[0.0],0
4,2753,2004031,NaN,TAG Heuer Men's Special Edition Heuer Monaco Watch,"TAG Heuer Monaco Stainless Steel Blue Mens Watch CAW211P.FC6356 is befittingly iconic, dare to be non-traditional with this innovative timepiece. Defying the traditional looking round shaped wristwatch TAG Heuer introduced the Monaco collection featuring a contrasting yet distinctive square water resistant case. This pioneering collection presented the worlds first square automatic chronograph. Timepieces in this series encapsulates the power and speed of a motor racing car. Indeed the Monaco still remains a truly remarkable collection and audacious in its own right.",5250,GBP,NaN,556904,6187863,...,EUR,NaN,793378,2004031#6187863,0,False,"[3.8281166553, -4.2899880409]",0,[0.0],0


In [17]:
# create a new column for the chatbot response
df['chatbot_response_raw'] = ''
df['chatbot_question'] = ''

In [18]:
# loop through the dataframe and send the messages to the chatbot
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    message = f"Are these two products the same? product 1: {row['title_left']}, product 2: {row['title_right']}"
    error = ""
    
    try:
        response = getanswer(message)
    except:
        error = "Error: response was not set successfully"
        print(error)
        response = ""
    print(response)

    if response:
        # your code here
        df.at[index, 'chatbot_response_raw'] = response
        df.at[index, 'chatbot_question'] = message
    else:
        df.at[index, 'chatbot_response_raw'] = error
        df.at[index, 'chatbot_question'] = message
        print("Error: response was not set successfully")

  0%|          | 0/100 [00:00<?, ?it/s]

gptj_model_load: loading model from '/Users/aaronsteiner/Downloads/ggml-gpt4all-j-v1.2-jazzy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 4505.45 MB
gptj_model_load: memory_size =   896.00 MB, n_mem = 57344
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285
gptj_generate: seed = 42069
gptj_generate: number of tokens in prompt = 57

Are these two products the same? product 1: Brother HL-L6200DW Wireless High Speed Mono Laser 2-sided printing 250 sheet Network 46PPM, product 2: Brother HL-L6300DW Laser Printer - Monochrome - Duplex 200-sheet Network 46PPM. Both products have printing speeds of up to 46 pages per minute and are monochrome printers. However, there are a few differences. The first pr

In [19]:
# get the current date and time
now = datetime.now()

# save the dataframe as a json file
df.to_json(f'data/results/gpt4all/{now}_simple_promt.json')

In [25]:
def clean_response(response):
    if "yes" in response.lower():
        return 1
    elif "no" in response.lower():
        return 0
    else:
        return -1

In [32]:
# add a coloumn with a clean version of the chatbot response if the chatresponse contains Yes/yes it will be 1 else if it contains No/no it will be 0 if it is 0 or 1 
df['chatbot_response_clean'] = df['chatbot_response_raw'].apply(lambda x: clean_response(x))
df.head()

# Check the accuracy of the chatbot 
df[df['chatbot_response_clean'] == df['label']].shape[0]
print(f"Accuracy: {df[df['chatbot_response_clean'] == df['label']].shape[0] / df.shape[0]}")
# Check the number of -1 responses
df[df['chatbot_response_clean'] == -1].shape[0]

Accuracy: 0.28


43

In [22]:
df.head()

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,...,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response_raw,chatbot_question,chatbot_response_clean
0,1639,58378163,NaN,Brother HL-L6200DW Wireless High Speed Mono Laser 2-sided printing 250 sheet Network 46PPM,"WIRELESS HIGH SPEED MONO LASER PRINTER WITH2-Sided PRINTING FOR HIGH VOLUME USAGE(46 PPM, 520 Sheets Paper Tray, Built-in Network & WiFi)A monochrome printer with great features• Print using built-in wireless, Gigabit Ethernet networking or locally via USB interface.• Prints up to 48 pages per minute at up to 1200 x 1200 dpi resolution.• 520-sheet capacity paper tray adjusts for letter or legal size paper.50-sheet multi-purpose tray is for envelopes or alternative media types.• Save paper with automatic duplex printing (double-sided).• Advanced security restricts unauthorized printing and protects sensitive documents.• Easy to use control panel with backlit LCD display for easy",617.38,AUD,NaN,5712817,44664998,...,58378163#44664998,0,True,"[3.8065164089, -4.2674832344]",0,[0.0],0,"200-sheet Network 46PPM. Both products have printing speeds of up to 46 pages per minute and are monochrome printers. However, there are a few differences. The first product has a mono printing option while the second product does not. Additionally, the first product is wireless while the second product is not. Therefore, these two products are not the same.<|endoftext|>","Are these two products the same? product 1: Brother HL-L6200DW Wireless High Speed Mono Laser 2-sided printing 250 sheet Network 46PPM, product 2: Brother HL-L6300DW Laser Printer - Monochrome - Duplex",0
1,460,91664569,NaN,Double Socket Arm - Long - RAM-B-201U-C,"Long Arm is 6\"" for applications requiring more reach.",23.49,USD,NaN,1366684,17589843,...,91664569#17589843,0,True,"[3.8207249641, -4.2826638222]",0,[0.0],0,"/DVD Combo.\nNo, these are two completely different products. The first product is a Double Socket Arm Long RAM-B-201U-C, which is likely a gaming controller or joystick. The second product is a Pokemon S&S2: Rebel Clash BD/DVD Combo, which is likely a game console or gaming system.<|endoftext|>","Are these two products the same? product 1: Double Socket Arm - Long - RAM-B-201U-C, product 2: Pokemon S&S2: Rebel Clash BD",0
2,1760,1575940,NaN,Corsair Vengeance LPX RAM 16GB 2400MHz DDR4 UDIMM 288 Pin Desktop Memory,"CORSAIR Vengeance LPX 16GB DDR4 memory is designed for high-performance overclocking. Built from an custom performance PCB and highly-screened memory ICs.The heatspreader is made of pure aluminum for faster heat dissipation and the custom performance PCB helps manage heat and provides superior overclocking headroom.Vengeance modules are compatibility-tested across DDR4 systems for reliably fast performance. The small form factor makes it ideal for smaller cases or any system where internal space is at a premium.Support XMP 2.0, one setting is all it takes to automatically adjust to the fastest safe speed for your Vengeance LPX kit.~section~High-performance OverclockingVENGEANCE LPX memory is designed",5499.00,INR,NaN,150984,31345669,...,1575940#31345669,1,False,"[-4.3838095665, 4.6694221497]",1,[1.0],1,"Layer, 16GB Corsair Vengeance LPX ram 16GB DDR4-2400 MHz UDIMM-288 Pin Desktop Memory.<|endoftext|>","Are these two products the same? product 1: Corsair Vengeance LPX RAM 16GB 2400MHz DDR4 UDIMM 288 Pin Desktop Memory, product 2: 16GB Corsair Vengeance LPX schwarz DDR4-2400 DIMM CL14 Single",0
3,4145,7125489,NaN,"HIK 8CH TURBO HD DVR, DS-7208HUHI-K1","Hikvision 8-Channel Turbo HD DVR, 1080p, 3MP, 5MP, 8MP, BNC Spot out, VGA/HDMI, up to 16ch IP",0.00,NaN,NaN,1069672,50492036,...,7125489#50492036,0,True,"[3.8441700935, -4.3036942482]",0,[0.0],0,"- Product 1 is a high definition television receiver and is equipped with a USB connectio

In [23]:
# unique value counts of the chatbot response
df['chatbot_response_clean'].value_counts()

 0    52
-1    30
 1    18
Name: chatbot_response_clean, dtype: int64